# Индексация и эмбеддинги: Qdrant + OpenAI

В этом ноутбуке выполняются два шага:

1. Создание/проверка коллекции в Qdrant (локально, сервис уже запущен в Docker)
2. Чанкинг из `data/processed/sections` → генерация эмбеддингов (OpenAI) → загрузка в Qdrant

Параметры размера чанков и перекрытия берутся из `configs/ingest.yaml`. Эмбеддинги — `text-embedding-3-small` (требуется `OPENAI_API_KEY`).


## Шаг 1. Создание/проверка коллекции в Qdrant

На этом шаге создаём (или убеждаемся, что существует) коллекция `dnd_rule_assistant` с размерностью 1536 (подходит для `text-embedding-3-small`). Docker уже запущен — отдельная команда запуска не требуется.


In [ ]:
# Подключение к Qdrant и создание коллекции
import os
import sys
from pathlib import Path

# Поиск корня проекта
start = Path.cwd().resolve()
repo_root = start
for p in [start] + list(start.parents):
    if (p / 'src').is_dir() and (p / 'data').is_dir():
        repo_root = p
        break

# PYTHONPATH для модулей
sys.path.append(str(repo_root / 'src'))

from dnd_rag.providers.vectorstore import get_client, ensure_collection  # type: ignore

QDRANT_HOST = os.environ.get("QDRANT_HOST", "localhost")
QDRANT_PORT = int(os.environ.get("QDRANT_PORT", "6333"))
COLLECTION = "dnd_rule_assistant"
DIM = 1536  # text-embedding-3-small

client = get_client(host=QDRANT_HOST, port=QDRANT_PORT)
ensure_collection(client, COLLECTION, vector_size=DIM)

{"host": QDRANT_HOST, "port": QDRANT_PORT, "collection": COLLECTION, "dim": DIM}


## Шаг 2. Чанкинг из секций и эмбеддинг в Qdrant

- Берём секции из `data/processed/sections/*.jsonl`.
- Разбиваем на чанки по параметрам из `configs/ingest.yaml` (только внутри секций).
- Генерируем эмбеддинги OpenAI (`text-embedding-3-small`) и загружаем в коллекцию `dnd_rule_assistant`.

В эмбеддинг идёт только поле `text`; метаданные (`book_title`, `chapter_title`, `section_path`, `chunk_index`, `chunk_id`) пишем в payload Qdrant.


In [1]:
# Чанкинг из секций → эмбеддинги → загрузка в Qdrant
import os
import sys
from pathlib import Path
import json

# Поиск корня проекта
start = Path.cwd().resolve()
repo_root = start
for p in [start] + list(start.parents):
    if (p / 'src').is_dir() and (p / 'data').is_dir():
        repo_root = p
        break

# PYTHONPATH
sys.path.append(str(repo_root / 'src'))

from dnd_rag.core.config import DEFAULT_CONFIG_PATH  # type: ignore
from dnd_rag.core.pipelines import chunks_from_sections_pipeline  # type: ignore
from dnd_rag.providers.embeddings import embed_texts  # type: ignore
from dnd_rag.providers.vectorstore import get_client, ensure_collection, upsert_vectors  # type: ignore

SECTIONS_DIR = repo_root / 'data' / 'processed' / 'sections'
CHUNKS_DIR = repo_root / 'data' / 'processed' / 'chunks'
CONFIG = DEFAULT_CONFIG_PATH

# 1) Чанкинг
produced = chunks_from_sections_pipeline(SECTIONS_DIR, CHUNKS_DIR, config_path=CONFIG)

# 2) Загрузка чанков и подготовка эмбеддингов
rows = []
for fp in produced:
    with Path(fp).open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))

texts = [r.get('text', '') for r in rows]
ids = [r.get('chunk_id', '') for r in rows]
payloads = [
    {
        'chunk_id': r.get('chunk_id', ''),
        'book_title': r.get('book_title', ''),
        'chapter_title': r.get('chapter_title', ''),
        'section_path': r.get('section_path', []),
        'chunk_index': r.get('chunk_index', 0),
    }
    for r in rows
]

vectors = embed_texts(texts, model='text-embedding-3-small')

# 3) Upload в Qdrant
QDRANT_HOST = os.environ.get('QDRANT_HOST', 'localhost')
QDRANT_PORT = int(os.environ.get('QDRANT_PORT', '6333'))
COLLECTION = 'dnd_rule_assistant'
DIM = 1536

client = get_client(host=QDRANT_HOST, port=QDRANT_PORT)
ensure_collection(client, COLLECTION, vector_size=DIM)
upsert_vectors(client, COLLECTION, ids=ids, vectors=vectors, payloads=payloads)

{'chunks_files': [str(p) for p in produced], 'total_chunks': len(rows)}


c:\Users\Shchurov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-11-16 23:59:19,393 - INFO - Loading dictionaries from c:\Users\Shchurov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymorphy2_dicts_ru\data
2025-11-16 23:59:19,431 - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


[CFG] ingest.yaml: C:\Users\Shchurov\Хранилище\Документы\05_Программирование\02_projects\personal\dnd_rule_assistant\configs\ingest.yaml (exists=True)


2025-11-16 23:59:29,868 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-16 23:59:35,783 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-16 23:59:41,031 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-16 23:59:43,234 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-11-16 23:59:43,242 - INFO - HTTP Request: GET http://localhost:6333/collections/dnd_rule_assistant/exists "HTTP/1.1 200 OK"
2025-11-16 23:59:43,782 - INFO - HTTP Request: PUT http://localhost:6333/collections/dnd_rule_assistant/points?wait=true "HTTP/1.1 200 OK"
2025-11-16 23:59:44,229 - INFO - HTTP Request: PUT http://localhost:6333/collections/dnd_rule_assistant/points?wait=true "HTTP/1.1 200 OK"
2025-11-16 23:59:44,665 - INFO - HTTP Request: PUT http://localhost:6333/collections/dnd_rule_assistant/points?wait=true "HTTP/1.1 200 OK"
2025-11-16 23:59:45,263 - INFO - HTTP Requ

{'chunks_files': ['C:\\Users\\Shchurov\\Хранилище\\Документы\\05_Программирование\\02_projects\\personal\\dnd_rule_assistant\\data\\processed\\chunks\\DMG.jsonl'],
 'total_chunks': 1519}